In [3]:
import numpy as np
import cv2
import  matplotlib.pyplot as plt
import os

In [19]:
root_dir = os.path.abspath("../")

def load_and_process_image(path):
    images = []
    smoothImages = []

    #load image
    img = cv2.imread(root_dir+path)
    images.append(img)
    # histogram equalization on image
    #img_eq = histogram_equalization(img)
    #final_image = cv2.hconcat([img,img_eq])
    #images.append(img_eq)
    # apply gabor filters on equalized image
    filters, filterValues = generate_gabor_filters()

    for x in range(len(filters)):
        fimg = cv2.filter2D(img,cv2.CV_8UC3,filters[x]) #img_eq

        n=13; #where n*n is the size of filter
        smoothed_image = cv2.medianBlur(fimg, n)
        smoothImages.append(smoothed_image)

        mg_eq = histogram_equalization(smoothed_image)
        images.append(mg_eq)
        #final_image= cv2.hconcat([final_image,fimg])

    final_image = h_concatenate_images(images)
    return images, smoothImages, final_image, filterValues


def histogram_equalization(img_in):
    # segregate color streams
    #image = cv2.imread(img_in)
    #b,g, r = img_in
    b,g,r =  cv2.split(img_in)
    h_b, bin_b = np.histogram(b.flatten(), 256, [0, 256])
    h_g, bin_g = np.histogram(g.flatten(), 256, [0, 256])
    h_r, bin_r = np.histogram(r.flatten(), 256, [0, 256])
# calculate cdf
    cdf_b = np.cumsum(h_b)
    cdf_g = np.cumsum(h_g)
    cdf_r = np.cumsum(h_r)

# mask all pixels with value=0 and replace it with mean of the pixel values
    cdf_m_b = np.ma.masked_equal(cdf_b,0)
    cdf_m_b = (cdf_m_b - cdf_m_b.min())*255/(cdf_m_b.max()-cdf_m_b.min())
    cdf_final_b = np.ma.filled(cdf_m_b,0).astype('uint8')

    cdf_m_g = np.ma.masked_equal(cdf_g,0)
    cdf_m_g = (cdf_m_g - cdf_m_g.min())*255/(cdf_m_g.max()-cdf_m_g.min())
    cdf_final_g = np.ma.filled(cdf_m_g,0).astype('uint8')
    cdf_m_r = np.ma.masked_equal(cdf_r,0)
    cdf_m_r = (cdf_m_r - cdf_m_r.min())*255/(cdf_m_r.max()-cdf_m_r.min())
    cdf_final_r = np.ma.filled(cdf_m_r,0).astype('uint8')
# merge the images in the three channels
    img_b = cdf_final_b[b]
    img_g = cdf_final_g[g]
    img_r = cdf_final_r[r]

    img_out = cv2.merge((img_b, img_g, img_r))
# validation
   #equ_b = cv2.equalizeHist(b)
   #equ_g = cv2.equalizeHist(g)
   #equ_r = cv2.equalizeHist(r)
   #equ = cv2.merge((equ_b, equ_g, equ_r))
    #print(equ)
    #cv2.imwrite('output_name.png', equ)
    return img_out
    #cv2.imshow(str(img),img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

def generate_gabor_filters():

    k = (50,50)
    sigma = 3
    theta = 1*np.pi/4
    lamda = 1*np.pi/4  #1/4 works best for angled.
    #gamma = .78 #.5 and .78 yields results
    gammArr = [.1,0.25,0.5,1.0]
    phi = 0
    filters = []
    values = []
    for x in range(1,4,2):
        theta = x*np.pi/4
        for y in gammArr:
            gamma = y
            kernel = cv2.getGaborKernel(k, sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
            #kernel = cv2.resize(kernel, (32,32))
            value = ("Sigma", sigma, "Theta", theta, "Gamma", gamma)
            values.append(value)
            filters.append(kernel)
            print(sigma,theta, gamma)

    return filters, values

def h_concatenate_images(imgArr):
    _img = imgArr[0]
    for x in range(len(imgArr)-1):
        _img = cv2.hconcat([_img,imgArr[x+1]])
    return _img

def v_concatenate_images(imgArr):
    _img = imgArr[0]
    for x in range(len(imgArr)-1):
        _img = cv2.vconcat([_img,imgArr[x+1]])
    return _img

def pairs_from_array(arr1,arr2):
    out=[]
    arr1 = arr1[1:]
    for x in range(len(arr1)):
        im1 = arr1[x]
        im2 = arr2[x]
        _img = cv2.hconcat([im1,im2])
        out.append(_img)
    return  out


smoothed_eq_img_arr, smt_arr, f_img, f_values = load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--BFP.tif")
smoothed_eq_img_arr1, smt_arr1, fimg1, f_values1 = load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--DsRed.tif")
smoothed_eq_img_arr2, smt_arr2, fimg2, f_values2 = load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--eGFP.tif")
smoothed_eq_img_arr3, smt_arr3, fimg3, f_values3 = load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--miRFP.tif")

pairs = pairs_from_array(smoothed_eq_img_arr, smt_arr)

for x in range(len(pairs)):
    cv2.imshow("Pair" +  str(f_values[x]) , pairs[x])
    cv2.imwrite("Pair_eq" + str(f_values[x]) + ".jpeg", pairs[x])

#cv2.imwrite("filtertest_01_noFilter.jpg",  f_img)
#FINAL_IMAGE = cv2.vconcat([f_img,f_img1])
#FINAL_IMAGE = cv2.vconcat([FINAL_IMAGE,f_img2])
#FINAL_IMAGE = cv2.vconcat([FINAL_IMAGE,f_img3])

def show_filters():
    filters = generate_gabor_filters()
    return h_concatenate_images(filters)




#cv2.imshow("Filters",cv2.resize(show_filters(),(800,100)))
#cv2.imshow(str(f_img),cv2.resize(f_img,(1000,100)))
cv2.waitKey(0)
cv2.destroyAllWindows()

3 0.7853981633974483 0.1
3 0.7853981633974483 0.25
3 0.7853981633974483 0.5
3 0.7853981633974483 1.0
3 2.356194490192345 0.1
3 2.356194490192345 0.25
3 2.356194490192345 0.5
3 2.356194490192345 1.0
3 0.7853981633974483 0.1
3 0.7853981633974483 0.25
3 0.7853981633974483 0.5
3 0.7853981633974483 1.0
3 2.356194490192345 0.1
3 2.356194490192345 0.25
3 2.356194490192345 0.5
3 2.356194490192345 1.0
3 0.7853981633974483 0.1
3 0.7853981633974483 0.25
3 0.7853981633974483 0.5
3 0.7853981633974483 1.0
3 2.356194490192345 0.1
3 2.356194490192345 0.25
3 2.356194490192345 0.5
3 2.356194490192345 1.0
3 0.7853981633974483 0.1
3 0.7853981633974483 0.25
3 0.7853981633974483 0.5
3 0.7853981633974483 1.0
3 2.356194490192345 0.1
3 2.356194490192345 0.25
3 2.356194490192345 0.5
3 2.356194490192345 1.0


In [5]:
import sys
sys.path.append('../Scripts/')


from Scripts.Utility import Utility2
# util import Utility

ut = Utility2()

ut.Test()

in init
HUH?


In [6]:
smoothed_eq_img_arr, smt_arr, f_img, f_values = ut.load_and_process_image("/Input/Images/--W00001--P00004--Z00000--T00000--BFP.tif")

pairs = ut.pairs_from_array(smoothed_eq_img_arr, smt_arr)

for x in range(len(pairs)):
    cv2.imshow("Pair" +  str(f_values[x]) , pairs[x])
    cv2.imwrite("Pair_eq" + str(f_values[x]) + ".jpeg", pairs[x])

#cv2.imwrite("filtertest_01_noFilter.jpg",  f_img)
#FINAL_IMAGE = cv2.vconcat([f_img,f_img1])
#FINAL_IMAGE = cv2.vconcat([FINAL_IMAGE,f_img2])
#FINAL_IMAGE = cv2.vconcat([FINAL_IMAGE,f_img3])

def show_filters():
    filters = generate_gabor_filters()
    return h_concatenate_images(filters)




#cv2.imshow("Filters",cv2.resize(show_filters(),(800,100)))
#cv2.imshow(str(f_img),cv2.resize(f_img,(1000,100)))
cv2.waitKey(0)
cv2.destroyAllWindows()

3 0.7853981633974483 0.1
3 0.7853981633974483 0.25
3 0.7853981633974483 0.5
3 0.7853981633974483 1.0
3 2.356194490192345 0.1
3 2.356194490192345 0.25
3 2.356194490192345 0.5
3 2.356194490192345 1.0


In [21]:
#get filters [1] and [5] and try to merge

filter1 = smoothed_eq_img_arr[1]
filter2 = smoothed_eq_img_arr[5]

mergedImage  = cv2.addWeighted(filter1, 0.5, filter2, .5, 0)
mergedImage1  = cv2.addWeighted(smoothed_eq_img_arr1[1], 0.5, smoothed_eq_img_arr1[5], .5, 0)
mergedImage2  = cv2.addWeighted(smoothed_eq_img_arr2[1], 0.5, smoothed_eq_img_arr2[5], .5, 0)
mergedImage3  = cv2.addWeighted(smoothed_eq_img_arr3[1], 0.5, smoothed_eq_img_arr3[5], .5, 0)
cv2.imshow("mergedImage",cv2.resize(mergedImage,(500,500)))
cv2.imshow("mergedImage1",cv2.resize(mergedImage1,(500,500)))
cv2.imshow("mergedImage2",cv2.resize(mergedImage2,(500,500)))
cv2.imshow("mergedImage3",cv2.resize(mergedImage3,(500,500)))


cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
GrayMerged = cv2.cvtColor(mergedImage,cv2.COLOR_BGR2GRAY)
GrayMerged1 = cv2.cvtColor(mergedImage1,cv2.COLOR_BGR2GRAY)
GrayMerged2 = cv2.cvtColor(mergedImage2,cv2.COLOR_BGR2GRAY)
GrayMerged3 = cv2.cvtColor(mergedImage3,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(GrayMerged,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
ret1, thresh1 = cv2.threshold(GrayMerged1,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
ret2, thresh2 = cv2.threshold(GrayMerged2,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
ret3, thresh3 = cv2.threshold(GrayMerged3,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [23]:
thresh = 255-thresh
thresh1 = 255-thresh1
thresh2 = 255-thresh2
thresh3 = 255-thresh3

In [29]:
mergedImages = [mergedImage, mergedImage1, mergedImage2, mergedImage3]
thresholds = [thresh,thresh1,thresh2,thresh3]
markersArr = []
markersApplied = []

# noise removal
kernel = np.ones((3,3),np.uint8)
for x in range(len(thresholds)):


    opening = cv2.morphologyEx(thresholds[x],cv2.MORPH_OPEN,kernel, iterations = 15)
    # sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=10)
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    dist_transform = cv2.normalize(dist_transform, dist_transform, 0, 1.0, cv2.NORM_MINMAX)
    ret, sure_fg = cv2.threshold(dist_transform,0.1*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)



#cv2.imshow("Input", cv2.resize(thresh,(500,500)))
#cv2.imshow("Unknown", cv2.resize(unknown,(500,500)))
#cv2.imshow("Opening", cv2.resize(opening,(500,500)))
#cv2.imshow("distTransform", cv2.resize(dist_transform,(500,500)))
##cv2.imshow( "background",cv2.resize(sure_bg,(500,500)))
#cv2.imshow("foreground", cv2.resize(sure_fg,(500,500)))
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Marker labelling

    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1
    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    markers = cv2.watershed(mergedImages[x],markers)
    markersArr.append(markers)
    mergedImages[x][markers == -1] = [255,0,0]
    markersApplied.append(mergedImages[x])
    cv2.imshow("foreground"+str(x), cv2.resize(mergedImages[x],(500,500)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
AllMarkedImages=ut.h_concatenate_images(markersApplied)
cv2.imshow("WaterShedSegmented" + str(x)+".jpeg",cv2.resize(AllMarkedImages,(2000,500)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
path= r"C:\Users\Jeppe\Documents\School\Med9\WholeCellSegmentation\Input\Images\--W00001--P00004--Z00000--T00000--Trans.tif"
tranlight= cv2.imread(path)

tranlight[markers==-1] = [255,0,0]
tranlight[markers==18] = [0,255,0]
tranlight[markers==6] = [0,0,255]
cv2.imshow("TransmittedLightMarkers", tranlight)
cv2.imwrite("TransmittedLightMarkers.jpg", tranlight)
cv2.waitKey(0)
cv2.destroyAllWindows()